In [1]:
import pandas as pd
import numpy as np

In [2]:
df_final = pd.read_csv('../raw_data/Excel_files/Margin_result_PE_analysis_PC2_20210226.csv')

In [3]:
print(df_final.head())

   Unnamed: 0   item_code  avg_conv_rate  last_price   last_date  \
0           0  1000012530       0.016220       99.99  2021-02-13   
1           1  1000005727       0.017079       99.99  2021-02-25   
2           2  1000007327       0.011193      209.99  2021-02-25   
3           3  1000114877       0.018526      119.99  2021-02-25   
4           4  1000117179       0.002919      319.99  2021-02-25   

   avg_unique_views      PE  PE_err      pc2        item_skey  price_down  \
0         66.081387 -0.0187  0.0059   44.680  247523697279378   98.010065   
1        170.099034 -0.0246  0.0044   40.729   71488185050378   98.010065   
2         46.738462 -0.0229  0.0049   71.500    9591881119337  205.831919   
3        269.909091 -0.0155  0.0029   77.859   25038272605281  117.614039   
4        104.033696 -0.0242  0.0072  142.550   34780482897946  313.653773   

   conv_rate_down  conv_rate_down_err  Delta_price  Margin_down  Margin_base  \
0        0.016226        1.914621e-06    39.3452

In [4]:
df_final = df_final[['item_code','last_price','price_down','Margin_rate_to_base','PE','PE_err','pc2']]

In [5]:
df_final['Margin_rate_to_base']=df_final['Margin_rate_to_base']*100

In [6]:

df_final.head()

,item_code,last_price,price_down,Margin_rate_to_base,PE,PE_err,pc2
0,1000012530,99.99,98.010065,0.0374,-0.0187,0.0059,44.680
1,1000005727,99.99,98.010065,0.0492,-0.0246,0.0044,40.729
2,1000007327,209.99,205.831919,0.0458,-0.0229,0.0049,71.500
3,1000114877,119.99,117.614039,0.0310,-0.0155,0.0029,77.859
4,1000117179,319.99,313.653773,0.0484,-0.0242,0.0072,142.550


In [7]:
df_final=df_final.rename(columns={"last_price": "actual_price", "price_down": "optimal_price"})

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.master('local').config('spark.driver.memory', '8g').appName('read_tables').getOrCreate()


In [9]:
df_d_item = spark.read.parquet('/Users/gabriele.sabato/PycharmProjects/raw_data/redshift_tables/d_item/*')


In [10]:
df_d_item.createOrReplaceTempView('d_item')

In [11]:
df_d_item.show()

+---------------+------------------+---------------------+-------------+----------------+----------+-------------+-----------+----------+-----------+--------------+-------------+--------------------+----------------------+--------------------+------------------+--------------------+------------------+-------------------+-------------------+---------+--------------------------------+---------------------------------------------+-----------------------+---------------------+------------------------+--------------------------+------------------------+--------------------+-----------+-------------------+------------------+-------------------+----------------------------+---------------+-----------------------+-----------------------+----------------------+------------------------------------+-------------------------------------------+-------------------------------------------------+--------------------------------------------------------+------------------------+---------------------+---

In [12]:
df_spark_final = spark.createDataFrame(df_final)


In [13]:
df_spark_final.createOrReplaceTempView('margin')

In [14]:
sp_df_pc2 = spark.read.parquet(
    '/Users/gabriele.sabato/PycharmProjects/raw_data/redshift_tables/f_item_status_last/*.parquet',
    header=True)
sp_df_pc2.show()

+-----------+---------+----------------+-----------------------+--------------------------+--------------------------+--------------------------+--------------------------------+---------------------------------+---------------------------------------+
|  item_skey|shop_skey|item_status_skey|item_status_weekly_skey|item_pc1_costs_before_cogs|item_pc2_costs_before_cogs|group_pc2_margin_simulated|group_pc2_margin_ratio_simulated|group_pc2_margin_simulated_pocket|group_pc2_margin_ratio_simulated_pocket|
+-----------+---------+----------------+-----------------------+--------------------------+--------------------------+--------------------------+--------------------------------+---------------------------------+---------------------------------------+
| 4184608796|        1|               4|                      4|                   674.169|                   800.340|                      null|                            null|                             null|                             

In [15]:
sp_df_pc2.createOrReplaceTempView('f_item_last_status')

In [16]:
df_d_item_table_margin = spark.sql("""SELECT    margin.*,
                                                di.item_parent_item_code, di.item_main_category, di.item_inventory_flag, di.item_gtin_comparable,di.item_exclusivity,di.item_skey,
                                                fils.item_pc1_costs_before_cogs as pc1_costs
                                                from margin
                                                INNER JOIN d_item as di on di.item_code = margin.item_code
                                                INNER JOIN f_item_last_status as fils on fils.item_skey=di.item_skey and fils.shop_skey=1
                                                """)

df_d_item_table_margin.show()

+----------+------------+------------------+--------------------+-------+------+------------------+---------------------+--------------------+-------------------+--------------------+----------------+---------------+---------+
| item_code|actual_price|     optimal_price| Margin_rate_to_base|     PE|PE_err|               pc2|item_parent_item_code|  item_main_category|item_inventory_flag|item_gtin_comparable|item_exclusivity|      item_skey|pc1_costs|
+----------+------------+------------------+--------------------+-------+------+------------------+---------------------+--------------------+-------------------+--------------------+----------------+---------------+---------+
|1000012530|       99.99| 98.01006534394246| 0.03740000000003154|-0.0187|0.0059|             44.68|   000000008000000774|ESSZIMMERSTÜHLE&B...|               laso|      non-comparable|       exclusive|247523697279378|   37.450|
|1000005727|       99.99| 98.01006534394246| 0.04920000000001229|-0.0246|0.0044|            

In [17]:
df_d_item_table_margin.createOrReplaceTempView('item_margin')

In [18]:
df_standalone_item = spark.sql("""SELECT *
                from item_margin
                where item_parent_item_code like '%-P' """)

#127 items

In [19]:
df_pd_sa_item=df_standalone_item.toPandas()

In [20]:
df_pd_sa_item['item_code'].head()


0    1000007327
1    1000117179
2    1000129145
3    1000104023
4    1000156992
Name: item_code, dtype: int64

In [21]:
item_sa_list = list(df_pd_sa_item['item_code'])

In [22]:
print(item_sa_list)

[1000007327, 1000117179, 1000129145, 1000104023, 1000156992, 1000182807, 1000065698, 1000153506, 1000054183, 1000092325, 1000111508, 1000123698, 1000051350, 1000118214, 1000008048, 1000123220, 1000050850, 1000150712, 1000008748, 1000119757, 1000065873, 1000110022, 1000117175, 1000118448, 1000007776, 1000008540, 1000110390, 1000047992, 1000145703, 1000015609, 1000033614, 1000007385, 1000048014, 1000054182, 1000123474, 1000129148, 1000039125, 1000022032, 1000022035, 1000027404, 1000114113, 1000164750, 1000008176, 1000123519, 1000135008, 1000008815, 1000169000, 1000183892, 1000098236, 1000181283, 1000007403, 1000176929, 1000112672, 1000116754, 1000116755, 1000182984, 1000008729, 1000104058, 1000197007, 1000050844, 1000007705, 1000034837, 1000119632, 1000005221, 1000170702, 1000008531, 1000065695, 1000015550, 1000021855, 1000150715, 1000123758, 1000154383, 1000127861, 1000129701, 1000120238, 1000131989, 1000015906, 1000008069, 1000008532, 1000008204, 1000050849, 1000200158, 1000008807, 100

In [23]:
pd_d_item_margin = df_d_item_table_margin.toPandas()




In [24]:
import psycopg2

def connect_redshift_db():
    cnx = psycopg2.connect(dbname= 'dwh',
                     host='redshift-prod.data.service.home24.net',
                     port= '5439',
                     user= 'gabriele_sabato',
                     password= 'zQZ8x6ZhRoTf7GS')
    return cnx
cnx_redshift = connect_redshift_db()
cursor_redshift = cnx_redshift.cursor()

In [25]:
def query_to_dataframe(_cursor, _query):
    _cursor.execute(_query)
    colnames = [desc[0] for desc in _cursor.description]
    print(colnames)
    return pd.DataFrame(_cursor.fetchall(), columns=colnames)

In [26]:
item_list=list(pd_d_item_margin['item_code'])
new_list = []
final_list = ""

for i in item_list:
    new_list.append(" '00000000" + str(i)+"' , " )

print(new_list)
final_list = final_list.join(new_list)
final_list=final_list[0:-3]

[" '000000001000012530' , ", " '000000001000005727' , ", " '000000001000007327' , ", " '000000001000114877' , ", " '000000001000117179' , ", " '000000001000176734' , ", " '000000001000008470' , ", " '000000001000129145' , ", " '000000001000012817' , ", " '000000001000022615' , ", " '000000001000104023' , ", " '000000001000156992' , ", " '000000001000182807' , ", " '000000001000008474' , ", " '000000001000008491' , ", " '000000001000022564' , ", " '000000001000174114' , ", " '000000001000061845' , ", " '000000001000065698' , ", " '000000001000110285' , ", " '000000001000153506' , ", " '000000001000022482' , ", " '000000001000054183' , ", " '000000001000173830' , ", " '000000001000184194' , ", " '000000001000016092' , ", " '000000001000092325' , ", " '000000001000111508' , ", " '000000001000138433' , ", " '000000001000006373' , ", " '000000001000012569' , ", " '000000001000080491' , ", " '000000001000123698' , ", " '000000001000016094' , ", " '000000001000016096' , ", " '0000000010000513

In [27]:
print(final_list)

 '000000001000012530' ,  '000000001000005727' ,  '000000001000007327' ,  '000000001000114877' ,  '000000001000117179' ,  '000000001000176734' ,  '000000001000008470' ,  '000000001000129145' ,  '000000001000012817' ,  '000000001000022615' ,  '000000001000104023' ,  '000000001000156992' ,  '000000001000182807' ,  '000000001000008474' ,  '000000001000008491' ,  '000000001000022564' ,  '000000001000174114' ,  '000000001000061845' ,  '000000001000065698' ,  '000000001000110285' ,  '000000001000153506' ,  '000000001000022482' ,  '000000001000054183' ,  '000000001000173830' ,  '000000001000184194' ,  '000000001000016092' ,  '000000001000092325' ,  '000000001000111508' ,  '000000001000138433' ,  '000000001000006373' ,  '000000001000012569' ,  '000000001000080491' ,  '000000001000123698' ,  '000000001000016094' ,  '000000001000016096' ,  '000000001000051350' ,  '000000001000118214' ,  '000000001000118866' ,  '000000001000005773' ,  '000000001000008048' ,  '000000001000117277' ,  '00000000100012

In [28]:
campaign_flag = """SELECT item_code,item_campaign_category, item_campaign_key, item_campaign_label
                    FROM dwh.public.item_catalog
                    WHERE item_code in ({FINAL_LIST})
                        and app_domain_id=1 """.format(FINAL_LIST=final_list)



In [29]:
df_campaign = query_to_dataframe(cursor_redshift, campaign_flag)



['item_code', 'item_campaign_category', 'item_campaign_key', 'item_campaign_label']


In [30]:
df_campaign.head()

,item_code,item_campaign_category,item_campaign_key,item_campaign_label
0,000000001000007385,2020061801,sidewow,EXTRA -12 %
1,000000001000007717,,garden12,EXTRA -12 %
2,000000001000008770,2020061801,sidewow,EXTRA -12 %
3,000000001000008815,,,
4,000000001000012754,,crm-skandideluxe,crm


In [31]:
item_skey_list = list(pd_d_item_margin['item_skey'])
string_ints = [str(int) for int in item_skey_list]
str_of_ints = ",".join(string_ints)
print(str_of_ints)



247523697279378,71488185050378,9591881119337,25038272605281,34780482897946,38255991794505,66003517992621,92807562060189,24436568574430,74462021394914,20953685067427,69450844745155,100679798534570,154928563530175,73662191637457,196145403809982,169682504248479,2692801778218,135975807930039,108943076297895,246252078866332,138994088712555,46842188366232,120211027959520,200966160233429,20395642409368,127226683695048,20211508201120,128820719966683,122763093380873,187786938344533,186011930802648,108213988466089,141224110778088,151983822903626,198292212567208,216387891363420,229844485949139,114004594017798,138230763214441,208217315863442,129323200930102,234935292544715,139976088621728,112618974590311,119918308738011,42654719757498,2066942130973,146728563835247,112186534043649,260759150426570,228313132843940,220427820739992,187990528223809,157121327029411,159724038661318,170043872779977,206469256249904,175131193104788,99929938066990,237753328512171,172434730983905,128892152043482,27585341305857

In [32]:
last_price_rule = """
SELECT item_skey, item_final_price_rule, period_date_skey--item_final_price_rule
  from dwh.star_analytical.f_item_attribute_history
 where shop_skey = 1
   and period_date_skey = (SELECT MAX(period_date_skey)
                             FROM dwh.star_analytical.f_item_attribute_history)
   and item_skey in ({ITEM_SKEY})
""".format(ITEM_SKEY=str_of_ints)

In [33]:
df_last_price_rule = query_to_dataframe(cursor_redshift, last_price_rule)

['item_skey', 'item_final_price_rule', 'period_date_skey']


In [34]:
sp_last_price_rule = spark.createDataFrame(df_last_price_rule)
sp_last_price_rule.createOrReplaceTempView('last_price_rule')
sp_campaign = spark.createDataFrame(df_campaign)
sp_campaign.createOrReplaceTempView('campaign')

In [35]:
final_df_margin = spark.sql("""SELECT im.*,
                    lpr.item_final_price_rule,
                    camp.item_campaign_key,
                    camp.item_campaign_label
                    from item_margin as im
                    INNER JOIN last_price_rule as lpr on lpr.item_skey = im.item_skey
                    INNER JOIN campaign as camp on CAST(camp.item_code as int) = im.item_code
                    """)

In [36]:
final_df_margin.show()

+----------+------------+------------------+--------------------+--------------------+------+------------------+---------------------+--------------------+-------------------+--------------------+------------------+---------------+---------+---------------------+------------------+--------------------+
| item_code|actual_price|     optimal_price| Margin_rate_to_base|                  PE|PE_err|               pc2|item_parent_item_code|  item_main_category|item_inventory_flag|item_gtin_comparable|  item_exclusivity|      item_skey|pc1_costs|item_final_price_rule| item_campaign_key| item_campaign_label|
+----------+------------+------------------+--------------------+--------------------+------+------------------+---------------------+--------------------+-------------------+--------------------+------------------+---------------+---------+---------------------+------------------+--------------------+
|1000007776|      269.99|262.01058960276174| 0.09539999999997688|             -0.0318|0.

In [37]:
df_margin_panda= final_df_margin.toPandas()


In [38]:
df_wow = df_margin_panda[df_margin_panda['item_campaign_key'].str.contains('wow')]

In [39]:
print(len(df_wow['item_code']))

291


In [40]:
df_not_wow = df_margin_panda[~df_margin_panda['item_campaign_key'].str.contains('wow')]

In [41]:
print(len(df_not_wow))

316


In [42]:
df_not_wow_sa = df_not_wow[df_not_wow['item_parent_item_code'].str.contains('-P')]

In [43]:
print(len(df_not_wow_sa))

122


In [44]:
df_sa = df_margin_panda[df_margin_panda['item_parent_item_code'].str.contains('-P')]

In [45]:
print(df_sa['item_code'].nunique())

221


In [46]:
print(len(df_sa['item_code'].index))

221


In [47]:

df_margin_panda.to_csv('../raw_data/Excel_files/Margin_result_all_in_20210226.csv')

In [48]:
df_sa.to_csv('../raw_data/Excel_files/Margin_result_sa_20210226.csv')


In [49]:
shuffled = df_sa.sample(frac=1)
result = np.array_split(shuffled, 3)

two_third = pd.concat([result[0], result[1]])

two_third.to_csv('../raw_data/Excel_files/Margin_result_sa_test_20210226.csv')
result[2].to_csv('../raw_data/Excel_files/Margin_result_sa_control_20210226.csv')

